In [1]:
%pip install z3-solver

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Codificação lógica de um programa
Considere o seguinte programa C sobre inteiros.

```
z = 0;
x = x + y;
if (y >= 0) {
  y = x - y;
  x = x - y;
}
else {
  z = x - y;
  x = y;
  y = 0;
}
z = x + y + z;
```
Faça a codificação lógica deste programa.


Conversão para SA(single-assignment form):
z1 = 0;
x1 = x0 + y0;
if (y0 >= 0){
    y1 = x1 - y0;
    x2 = x1 - y1;
}
else {
    z2 = x1 - y0;
    x2 = y0;
    y1 = 0;
}
x3 = y0 >= 0 ? x2 : x2;
y2 = y0 >= 0 ? x1 : z1;
z3 = y0 >= 0 ? z1 : x0;

z4 = x3 + y2 + z3;

Conversão para CNF(conditional normal form):
z1 = 0;
x1 = x0 + y0;
if(y0 >= 0) y1 = x1 - y0;x2 = x1 - y1;
if(!(y0 >= 0)) z2 = x1 - y0;x2 = y0;y1 = 0;
x3 = y0 >= 0 ? x2 : x2;
y2 = y0 >= 0 ? x1 : z1;
z3 = y0 >= 0 ? z1 : x0;

z4 = x3 + y2 + z3;

Codificação lógica do programa em Z3:

In [33]:
from z3 import *

Z1,X1,X0,Y0,Y1,X2,Z2,Z4,X3,Y2,Z3 = Ints('Z1 X1 X0 Y0 Y1 X2 Z2 Z4 X3 Y2 Z3')
# Criação do Solver
s = Solver()
# Restrição à programa 
s.add(Z1 == 0)
s.add(X1 == X0 + Y0)

s.add(Implies(Y0 >= 0,Y1 == X1 - Y0))
s.add(Implies(Y0 >= 0,X2 == X1 - Y1))

s.add(Implies(Not(Y0 >= 0),Z2 == X1-Y0))
s.add(Implies(Not(Y0 >= 0),X2 == Y0))
s.add(Implies(Not(Y0 >= 0),Y1 == 0))



s.add(Implies(Y0 >= 0,X3 == X2))
s.add(Implies(Not(Y0 >= 0),X3 == X2))

s.add(Implies(Y0 >= 0,Y2 == X1))
s.add(Implies(Not(Y0 >= 0),Y2 == Z1))

s.add(Implies(Y0 >=0, Z3 == Z1))
s.add(Implies(Not(Y0 >= 0),Z3 == X0))


s.add(Z4 == X3 + Y2 + Z3)


if s.check() == sat :
    print("Ok.")
else:
    print("A codificação não deve estar bem.")

Ok.


Tendo por base a codificação lógica que fez do programa, utilize o API do Z3 para Python para se pronunciar quanto à veracidade das seguintes afirmações. Justifique a sua resposta. No caso da afirmação ser falsa, apresente o contra-exemplo indicado pelo solver.

   1. *Se o valor inicial de $y$ for positivo, o programa faz a troca dos valores de $x$ e $y$ entre si.*

   2. *O valor final de $y$ nunca é negativo.*
   
   3. *O valor final de $z$ corresponde à soma dos valores de entrada de $x$ e $y$.*

   4. *O valor final de x é sempre negativo.*

In [24]:
s.push()

prop = "\n\"Se o valor inicial de y for positivo, o programa faz a troca \
dos valores de x e y entre si.\""

s.add(Not(Implies(Y0 >= 0, And(Y1 == X0, X2 == Y0))))
if (s.check() == unsat):
    print("A afirmação é verdadeira.")

else:
    print("A afirmação é falsa ")
    print(s.model())

s.pop()

A afirmação é verdadeira.


In [25]:
s.push()

prop = "\n\"O valor final de y nunca é negativo.\""

s.add(Not(Y2 >= 0))
if (s.check() == unsat):
    print("A afirmação é verdadeira.")
    print(s.model())
else:
    print("A afirmação é falsa. ")
    print(s.model())

s.pop()

A afirmação é falsa. 
[Z1 = 0,
 X3 = 1,
 X2 = 1,
 X1 = -1,
 Y2 = -1,
 X0 = -2,
 Z3 = 0,
 Y0 = 1,
 Y1 = -2,
 Z4 = 0]


In [30]:
s.push()

prop = "\n\"O valor final de z corresponde à soma dos valores de entrada de x e y.\""

s.add (Not(Z4 == X0 + Y0))
if (s.check() == unsat):
    print("A afirmação é verdadeira")
else:
    print("A afirmação é falsa")
    print(s.model())

s.pop()

A afirmação é falsa
[Z1 = 0,
 X3 = 1,
 X2 = 1,
 X1 = -1,
 Y2 = -1,
 X0 = -2,
 Z3 = 0,
 Y0 = 1,
 Y1 = -2,
 Z4 = 0]


In [32]:
s.push()

prop = "\n\"O valor final de x é sempre negativo.\""

s.add (Not(X3 < 0))
if (s.check() == unsat):
    print("A afirmação é verdadeira ")
else:
    print("A afirmação é falsa")
    print(s.model())

s.pop()

A afirmação é falsa
[Z1 = 0,
 X3 = 1,
 X2 = 1,
 X1 = -1,
 Y2 = -1,
 X0 = -2,
 Z3 = 0,
 Y0 = 1,
 Y1 = -2,
 Z4 = 0]
